# Training machine learning models

## Loading training and validation datasets

In [1]:
from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split

ROOT = Path("__file__").resolve().parent
DATA = ROOT / 'data'
MODELS = ROOT / 'models'
TRAIN_AND_VAL_DS = DATA / 'US_2023_JUL_25_complete_cases_reviewd.csv'
TEST_DS = DATA / 'US_2023_AGO_01_all_cases_validationset.csv'

train_and_val_ds = pd.read_csv(TRAIN_AND_VAL_DS)
test_ds = pd.read_csv(TEST_DS)

retrospective = train_and_val_ds['study'] == 'retrospective'
prospective = train_and_val_ds['study'] == 'prospective'
birads_4a = train_and_val_ds['birads'] == '4a'
birads_4b = train_and_val_ds['birads'] == '4b'
prospective_4ab = prospective & (birads_4a | birads_4b)

extra4_ds, val_ds = train_test_split(train_and_val_ds[prospective_4ab], train_size=0.2, random_state=7)

train_ds = pd.concat([train_and_val_ds[retrospective], extra4_ds])

train_ds['birads'].value_counts()

birads
4a    407
5     225
4b    204
4c    184
3      44
6      18
Name: count, dtype: int64

In [2]:
val_ds['birads'].value_counts()

birads
4a    170
4b     62
Name: count, dtype: int64

In [3]:
test_ds['birads'].value_counts()

birads
4A    130
5      88
4C     71
6      55
4B     55
3       4
Name: count, dtype: int64

In [4]:
X_train = train_ds.drop(columns=['birads', 'result', 'study'])
y_train = train_ds['result']

X_val = val_ds.drop(columns=['birads', 'result', 'study'])
y_val = val_ds['result']

X_test = test_ds.drop(columns=['pt_id', 'img_id', 'birads', 'result', 'study'])
y_test = test_ds['result']

## Hyperparameter tuning

In [ ]:
from joblib import dump

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from .models import pipelines, param_grids

best_clfs = {}

for name, pipe in pipelines.items():
    clf = None
    if name == 'XB' or name == 'RF':
        clf = RandomizedSearchCV(pipe,
                                 param_distributions=param_grids[name],
                                 n_iter=5,
                                 scoring='accuracy',
                                 n_jobs=4,
                                 cv=10,
                                 verbose=3)
    else:
        clf = GridSearchCV(pipe,
                           param_grid=param_grids[name],
                           scoring='accuracy',
                           cv=10)
    clf.fit(X_train, y_train)
    best_clfs[name] = (clf.best_params_, clf.score(X_train, y_train))

dump(best_clfs, MODELS / 'res_hyperparameter_tuning.joblib')

## Selecting between the different models

In [5]:
from joblib import load

# best_clfs = load(MODELS / 'res_hyperparameter_tuning.joblib')

# temporary:
import pickle
from sklearn.pipeline import Pipeline
from models import ct, models

best_params = None
with open(MODELS / 'grid_search_dictionary_all_2023_5_24.pkl', 'rb') as f:
    best_params = pickle.load(f)

pipelines = {name: Pipeline([
    ('ct', ct),
    (name, model(**best_params[name][0][0]))
]) for name, model in models.items()}

for name, model in pipelines.items():
    print('Fitting {}'.format(name))
    model.fit(X_train, y_train)

Fitting SVM
Fitting RF
Fitting LR
Fitting DT
Fitting AB
Fitting XB


/home/alek/.local/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


Fitting KN
Fitting MLP
Fitting GB


/home/alek/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [6]:
from sklearn.metrics import classification_report

clf_df = pd.DataFrame(
    columns=['npv', 'sens', 'spec', 'ppv', 'acc', 'f1'], index=pipelines.keys())

for name, model in pipelines.items():
    report = classification_report(y_val,
                                   model.predict(X_val),
                                   labels=[1, 0],
                                   output_dict=True)
    clf_df.loc[name] = {
        'npv' : report['0']['precision'],
        'sens': report['1']['recall'],
        'spec': report['0']['recall'],
        'ppv' : report['1']['precision'],
        'acc' : report['accuracy'],
        'f1'  : report['1']['f1-score'],
    }

clf_df.sort_values('npv', ascending=False)

,npv,sens,spec,ppv,acc,f1
DT,0.970149,0.666667,0.911215,0.387097,0.892241,0.489796
XB,0.969388,0.666667,0.88785,0.333333,0.87069,0.444444
RF,0.969231,0.666667,0.883178,0.324324,0.866379,0.436364
AB,0.968912,0.666667,0.873832,0.307692,0.857759,0.421053
SVM,0.968586,0.666667,0.864486,0.292683,0.849138,0.40678
KN,0.967742,0.666667,0.841121,0.26087,0.827586,0.375
MLP,0.965686,0.611111,0.920561,0.392857,0.896552,0.478261
GB,0.963918,0.611111,0.873832,0.289474,0.853448,0.392857
LR,0.963542,0.611111,0.864486,0.275,0.844828,0.37931


In [105]:
import numpy as np
from sklearn.metrics.pairwise import rbf_kernel

ct = pipelines['SVM']['ct']
svc = pipelines['SVM']['SVM']
X = ct.transform(X_train)[svc.support_]
alpha = svc.dual_coef_.ravel()
gamma = svc.gamma
K = rbf_kernel(X, gamma=gamma)
Lambda = np.sqrt(np.dot(alpha, K@alpha))

def lower_bound(m, delta, npv):
    return npv - Lambda*np.sqrt(1.0/m) - 6*np.sqrt(np.log(2 / delta) / (2*m))

# lower_bound(120000, 0.1)
# np.dot(np.ones(331), alpha)
0.5*np.dot(alpha, K@alpha)

1323.201046549369

In [51]:
from cvxopt import matrix, solvers

Q = matrix(K)
p = matrix(-np.ones(331).astype('float'))
A = matrix((y_train.to_numpy()[svc.support_] * 2 - 1).reshape(1, 331).astype('float'))
b = matrix([0.0])
G = matrix(np.vstack([np.eye(331), -np.eye(331)]).astype('float'))
h = matrix(np.concatenate([svc.C * np.ones(331), np.zeros(331)]).astype('float'))
sol = solvers.qp(Q, p, G, h, A, b)


     pcost       dcost       gap    pres   dres
 0:  4.9721e+03 -4.2599e+04  5e+04  5e-14  2e-14
 1:  7.2139e+02 -1.1557e+03  2e+03  3e-14  4e-14
 2:  9.8016e+01 -1.2036e+02  2e+02  1e-14  2e-14
 3:  1.2077e+01 -1.9662e+01  3e+01  2e-15  6e-15
 4:  8.0911e-01 -3.6521e+00  4e+00  2e-15  2e-15
 5: -4.2853e-01 -9.9426e-01  6e-01  8e-16  7e-16
 6: -5.0331e-01 -5.3936e-01  4e-02  2e-16  3e-16
 7: -5.0404e-01 -5.1231e-01  8e-03  2e-16  2e-16
 8: -5.0642e-01 -5.1471e-01  8e-03  3e-16  2e-16
 9: -5.0883e-01 -5.1129e-01  2e-03  2e-16  2e-16
10: -5.0976e-01 -5.1003e-01  3e-04  2e-16  2e-16
11: -5.0985e-01 -5.0998e-01  1e-04  4e-16  2e-16
12: -5.0991e-01 -5.0991e-01  2e-06  2e-16  3e-16
13: -5.0991e-01 -5.0991e-01  2e-08  4e-16  3e-16
Optimal solution found.


In [56]:
alpha = np.array(sol['x']).ravel()
np.sqrt(np.dot(alpha, K@alpha))

1.0098588288423476

In [120]:
from sklearn.svm import SVC

clf = SVC(C = 0.0001, kernel='rbf', gamma=0.0001, tol=0.00001)
clf.fit(X_train, 2*y_train-1)
print(clf.score(X_val, 2*y_val-1))

alpha = clf.dual_coef_.ravel()
gamma = clf.gamma
X = X_train.to_numpy()[clf.support_]
K = rbf_kernel(X, gamma=gamma)
Lambda = np.sqrt(np.dot(alpha, K@alpha))

print(lower_bound(m=X_train.shape[0], delta=0.15, npv=0.9224))
classification_report(2*y_val - 1, clf.predict(X_val), output_dict=True)['-1']
# classification_report(2*y_train - 1, clf.predict(X_train), output_dict=True)['-1']

0.9224137931034483
0.7145779059603694


/home/alek/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alek/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alek/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 18.0}